In [1]:
##NOTE: This notebook will run properly ONLY on PYTHON 3 KERNEL

import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
%matplotlib inline
import seaborn as sns
sns.set()
from matplotlib.backends.backend_pdf import PdfPages
#from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib as mpl
import numpy as np #import for transposing
import math

In [2]:
pup_df = pd.read_csv('../Mongoose Modeling/Updated Census.csv', infer_datetime_format = True)

#pup_df["Date"] = pup_df.Date.apply(lambda: x, datetime.strptime(x,'%d-%b-%Y'))

pup_df = pup_df.drop(0)
pup_df = pup_df.rename(columns = {'Unnamed: 0': 'Instance'})
pup_df["Date"] = pd.to_datetime(pup_df["Date"])
pup_df.columns

Index(['Instance', 'Observation ID', 'Date', 'Time', 'Troop', 'Observers',
       'Location', 'Latitude ', 'Longitude', 'Rain Type', 'Land Use',
       'Temperature', 'Cloud Cover', 'Wind', 'Wilderness', 'Den Type',
       'Park Habitat', 'Troop Number', 'Complete Count?', 'Group Spread',
       'Number juveniles/ Sub-Adults', '# Pups', '# Pregnant',
       'Pregnant third', 'Identifiable Individuals', 'Number sick',
       'Illness Rank', 'Description of Signs', 'Number injured',
       'Description of Injuries', 'Feces', '# Feces', 'Urine',
       'Anal Gland Secr.', 'Comments', 'Mix', 'Troop Name', 'Split',
       'Columna1', 'Columna2', 'Columna3', 'AM/PM', 'Time up', 'Not seen',
       'Columna4'],
      dtype='object')

In [3]:
pups = pup_df[pup_df["# Pups"].isnull() == False]
pups = pups[pups["Troop Number"].isnull() == False]
pups["# Pups"].unique()

array(['M9/7', '2390', '1', '2', '4', '8 to 9', '7-Jan', '14', '17', '12',
       '11', '20', '22', '21', '16', '7', '15', '9', '5', '6', 'Yes', '3',
       '13', '23', '34', 'LOTS', '19', '8', '10'], dtype=object)

In [4]:
pups["Troop Number"].unique()

array(['1', '8', '15', '10', '34', '44', '20', '23', '30', '27', '17',
       '11', '22', '21', '29', '41', '45', '40', '16', '25', '14+9', '7+1',
       '9+2', '30+5', '20+7', '12+7', '12+6', '8+3', '56', '26', '67',
       '18', '28', '39', '3', '9', '13', '32', '6', '24', '14', '12', '19',
       '7', '35', '50', '53', '33', '5'], dtype=object)

In [5]:
pups.shape

(166, 45)

In [6]:
#Fixing clerical errors
pups.loc[117,'# Pups'] = 9 
pups.loc[123,'# Pups'] = 7
pups.loc[306,'# Pups'] = 6
pups.loc[307,'# Pups'] = 6
pups = pups.drop([2,14,649])
pups["# Pups"] = pd.to_numeric(pups["# Pups"])


##Fixing clerical errors

pups.loc[296,'Troop Number'] = 8
pups.loc[312,'Troop Number'] = 8
pups.loc[298,'Troop Number'] = 11
pups.loc[299,'Troop Number'] = 35
pups.loc[300,'Troop Number'] = 27
pups.loc[301,'Troop Number'] = 19
pups.loc[304,'Troop Number'] = 18
pups.loc[317,'Troop Number'] = 11
pups.loc[323,'Troop Number'] = 11
pups.loc[294,'Troop Number'] = 23

pups["Troop Number"] = pd.to_numeric(pups['Troop Number'])

pups.shape

(164, 45)

In [7]:
pups['# Pups'].max()

34

In [8]:
pups = pups.reset_index(drop = True)
wet = pd.DataFrame()
for i in range(len(pups)):
    if (pups.loc[int(i),'Date'].month < 5) | (pups.loc[int(i),'Date'].month > 10):
        wet = wet.append(pups.iloc[i])
        
wet.shape

(160, 45)

In [9]:
dry = pd.DataFrame()
for i in range(len(pups)):
    if (pups.loc[int(i),'Date'].month > 4) & (pups.loc[int(i),'Date'].month < 10):
        dry = dry.append(pups.iloc[i])
        
dry

,# Feces,# Pregnant,# Pups,AM/PM,Anal Gland Secr.,Cloud Cover,Columna1,Columna2,Columna3,Columna4,...,Split,Temperature,Time,Time up,Troop,Troop Name,Troop Number,Urine,Wilderness,Wind
23,NaN,NaN,1.0,AM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,CSL,NaN,40.0,NaN,NaN,NaN
47,NaN,NaN,3.0,AM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Safari Lodge,NaN,39.0,NaN,NaN,NaN
52,2.0,NaN,2.0,NaN,no,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8:45,NaN,WA,NaN,17.0,no,NaN,NaN


In [10]:
dry = dry.reset_index(drop = True)
rates = []
dates = []
for i in range(len(dry)):
    rates.append(dry.iloc[i]['# Pups']/dry.iloc[i]['Troop Number'])
    dates.append(dry.iloc[i]["Date"])
dry_birth = (sum(rates)/len(rates))

print('birth rate during the dry season: ',dry_birth)


birth rate during the dry season:  0.0731900452489
